# <center>Machine Learning Project</center>

** **
## <center>*03.12 - Ensemble*</center>

** **

The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639



In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib

## <span style="color:salmon"> 1. Import Dataset </span> 

In [8]:
# Import dataset
train_df = pd.read_csv('../preprocessed_data/train_data.csv', index_col="Claim Identifier")

/var/folders/qy/5yyqg46n3mbfdxyxng5xt18r0000gn/T/ipykernel_4846/3885449982.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../preprocessed_data/train_data.csv', index_col="Claim Identifier")


In [3]:
# Import dataset
test_df = pd.read_csv('../preprocessed_data/test_data.csv', index_col = 'Claim Identifier')

/var/folders/qy/5yyqg46n3mbfdxyxng5xt18r0000gn/T/ipykernel_4846/1197995875.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('../preprocessed_data/test_data.csv', index_col = 'Claim Identifier')


## <span style="color:salmon"> 2. Prepare Dataset </span> 

Define y as a target "Claim Injury Type Encoded" and X with all the other columns

In [4]:
X = train_df.drop(["Claim Injury Type Encoded"], axis = 1)
y = train_df["Claim Injury Type Encoded"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
label_encoders = joblib.load('/Users/franciscocapontes/Universidade/Machine learning/Project/Encoders/dic_4_encoding.json')

# Apply the pre-existing encoders to your categorical columns
for col, encoder in label_encoders.items():
    # Ensure the column exists in your current dataset
    if col in X_train.columns:
        # Transform both training and test data using the saved encoder
        X_train[col] = encoder.transform(X_train[col])
        X_test[col] = encoder.transform(X_test[col])

KeyError: 123

In [10]:
# XGBoost
xgb_model = XGBClassifier(num_class=8, 
        objective="multi:softmax",
        eval_metric="merror",
        verbose = 0)
xgb_model.fit(X_train, y_train)

# LightGBM
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)

# CatBoost
catboost_model = CatBoostClassifier(verbose=0)
catboost_model.fit(X_train, y_train)

# Gradient Boosting
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Accident Date: object, County of Injury: object, District Name: object, Zip Code: object

In [ ]:
# Define the base models
base_models = [
    ("xgb", xgb_model),
    ("lgbm", lgbm_model),
    ("catboost", catboost_model),
    ("gb", gb_model),
]

# Stacking Classifier
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator = XGBClassifier(eval_metric="logloss", use_label_encoder=False),
    cv=5  
)

# Train stacking model
stacking_model.fit(X_train, y_train)

# Evaluate
y_pred = stacking_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
# Get individual predictions
xgb_pred = xgb_model.predict_proba(X_test)
lgbm_pred = lgbm_model.predict_proba(X_test)
catboost_pred = catboost_model.predict_proba(X_test)
gb_pred = gb_model.predict_proba(X_test)

# Weighted average of probabilities
weights = [0.25, 0.25, 0.25, 0.25]  # Adjust weights if necessary
final_pred = (weights[0] * xgb_pred +
              weights[1] * lgbm_pred +
              weights[2] * catboost_pred +
              weights[3] * gb_pred)

# Convert probabilities to class predictions
final_class_pred = np.argmax(final_pred, axis=1)

print("Accuracy:", accuracy_score(y_test, final_class_pred))


In [ ]:
# Voting Classifier
voting_model = VotingClassifier(
    estimators=base_models,
    voting="soft"  # Use "hard" for majority voting or "soft" for probabilities
)

# Train and evaluate
voting_model.fit(X_train, y_train)
y_pred = voting_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))